In [ ]:
import numpy as np
import scipy
import networkx as nx
import plotly.graph_objects as go

# Learning and Networks

Some Important Questions
- Will society adapt to a unique belief in the long run on a given topic?
- How fast can/will such a convergence occur?
- Will society actually learn?
    - Will society use available information to analyze it properly?
    - Will the beliefs in the society converge to the truth?
- Are some people better at convincing others to share their beliefs?

## Bayesian/Observational Learning

### Common set-up
* Finitely many repeated actions
* Infinitely many discrete periods
* Agents observing not only their payoffs, but the actions and outcomes for everyone of their neigbours, too

### Bala Goyal 98
* $n$ players in an undirected component $g$
* Choose action A or B each period
   - A pays 1 for sure
   - B pays 2 with probability $p$ and 0 with probability $1‐p$
* Each period get a payoff based on choice
* Also observe neighbors’ choices
* Maximize discounted stream of payoffs
$E \left[ \sum_t δ^t \pi_{it} \right]$
* $p$ is unknown takes on finite set of values
* Optimal strategy
   - Play A if $p < \frac{1}{2}$
   - Play B if $p > \frac{1}{2}$

### Reinforcement learning approach for optimal behaviour

Initialize, for $x \in (A, B)$:
$$Q(x) \leftarrow 0$$
$$N(x)  \leftarrow 0$$
Loop forever:
$$X \leftarrow \begin{cases}
\argmax_{x\in(A,B)} Q(x), & \text{with probability 1 − $\varepsilon$ (breaking ties randomly)} \\
\text{a random action $x\in(A,B)$}, & \text{with probability $\varepsilon$}
\end{cases}$$
$$R \leftarrow outcome(X)$$
$$N(X) \leftarrow N(X) + 1$$
$$Q(X) \leftarrow Q(X) + \frac{1}{N(X)}
\left[R − Q(X)\right]$$

### Time for demo

In [ ]:
import net_learn_obs
from dash import Dash
app = Dash(__name__, meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}])
net_learn_obs.create_net_learn_app(app)
app.run()

### Drawbacks
- If $p$ is not exactly $\frac{1}{2}$, then with probability 1 there is a time such that all agents in a given component play just one action (and all play the same action) from that time onward.
- If B is the right action then play the right action if converge to it, but might not
- If A is the right action, then must converge to right action

### Limitations
- Homogeneity of actions and payoffs across players
- What if heterogeneity?
- Repeated actions over time
- Stationarity
- Networks are not playing role here!

## DeGroot Social Interaction Model
- Individuals $\{1, \dots, n\}$
- $G$ weighted directed network, stochastic matrix
- Start with beliefs, attitude, etc. $b_{i}(0) \in [0,1]$
- can also have these be vectors…
- Updating: 
$$b_{i}(t) = \sum_{j} G_{ij} b_{j}(t‐1)$$

In [ ]:
trans_matrix = np.array([[1/3, 1/3, 1/3], [1/2, 1/2, 0], [1/2, 0, 1/2]]) 
trans_matrix

In [ ]:
b = np.array([[0], [1], [0]])
b

In [ ]:
trans_matrix.dot(b).transpose().tolist()

In [ ]:
def degroot_sim(trans_matrix, init_vector, n=100):
    mtrx = trans_matrix.transpose()
    vctr = init_vector.transpose()
    results = vctr.tolist()
    for x in range(n):
        vctr = vctr.dot(mtrx)
        results.append(vctr.tolist()[0])

    results = np.array(results).transpose().tolist()

    data = [
        go.Scatter(
            x=list(range(n+1)),
            y=y,
            mode="lines"
        )
        for y in results
    ]
    return go.Figure(data=data)

In [ ]:
degroot_sim(trans_matrix, b, n=10)

In [ ]:
trans_matrix = np.array([[1/3, 1/3, 1/3], [1/2, 1/2, 0], [0, 1/4, 3/4]]) 
trans_matrix

In [ ]:
degroot_sim(trans_matrix, b, n=10)

### When is there convergence?

In [ ]:
trans_matrix1 = np.array([[0, 1/2, 1/2], [1, 0, 0], [0, 1, 0]]) 
trans_matrix1

In [ ]:
degroot_sim(trans_matrix1, b, n=20)

In [ ]:
trans_matrix2 = np.array([[0, 1/2, 1/2], [1, 0, 0], [1, 0, 0]]) 
trans_matrix2

In [ ]:
degroot_sim(trans_matrix2, b, n=20)

- $G$ converges if $\lim_{t\rightarrow \infty} G^t b$ exists for all $b$
- $G$ is aperiodic if the greatest common divisor of its cycle lengths is one

In [ ]:
nx.draw(nx.from_numpy_array(trans_matrix1,  create_using=nx.DiGraph))

In [ ]:
nx.draw(nx.from_numpy_array(trans_matrix2,  create_using=nx.DiGraph))

- Suppose $G$ is strongly connected.
    - $G$ is convergent if and only if it is aperiodic.
    - $G$ is convergent if and only if: $\lim G^t = (1,1,….,1)^Ts$ where $s$ is the unique lhs eigenvector with eigenvalue 1

### When is there a consensus?
- convergence is sufficient

In [ ]:
res = scipy.linalg.eig(trans_matrix1, left=True)
print(f"eigenvalues = {res[0]}")
print(f"eingenvectors = {res[1]}")

In [ ]:
indx = 0
s = res[1][:,indx]/res[1][:, indx].sum()
print(f"vector s = {s}")
print(f"G^t = {np.array([[1], [1], [1]]).dot(s.reshape(1,3))}")

we could validate by diretcly doing the multiplications

In [ ]:
xx = trans_matrix1
for _ in range(100):
    xx = xx.dot(trans_matrix1)

print(xx)
print(xx.dot(b))

### Who has influence?

#### What are Limiting beliefs?
- When group reaches a consensus, what is it?
- Who are the influential agents in terms of steering the limiting belief?
- Must be that the rows of $G^t$ converge to same thing since beliefs converge to same thing for all initial vectors
- Consensus converges to (normalized) eigenvector weighted sum of original beliefs.
$$\lim G^t b(0) = (1,1,….,1)^Tsb(0) = (1,1,….,1)^T \sum s_{i}b_{i}(0)$$

#### Who has Influence?
- $s_i = \sum_j G_{ji} s_j$
- High influence from being paid attention to by people with high influence.
- Related to eigenvector centrality.

In [ ]:
print(f"s = {s}")
print(f"G = {trans_matrix1}")
print(f"s.G = {s.dot(trans_matrix1)}")

### When is the influence accurate?


#### When is Information Aggregation Accurate:
- How does this depend on network structure?
- How does it depend on influence?
- How does it relate to speed of convergence?


#### Uncertainty Structure
- Suppose true state is $\mu$
- Agent $i$ sees $b_i(0) = \mu + \varepsilon_i$
- $\varepsilon_i$ has 0 mean and finite variance, bounded below and above,
- signal distributions may differ across agents, but are independent conditional on $\mu$

#### Wise Crowds
- Consider large societies
- If they pooled their information, they would have an accurate estimate of $\mu$
- For what sequences of societies indexed by $n$ does 
$$P\left(\lim_{t\rightarrow\infty} | b_{j}^n(t) – \mu| > \delta\right)\xrightarrow[n\rightarrow\infty]{} 0 \text{ for all } \delta, j?$$
- Let $\varepsilon_i$'s be independent, zero mean, and each have finite variance (bounded below). Then:
$$\plim_{n\rightarrow\infty} \sum_{i=1}^{n} s_i^n \varepsilon_i = 0 \text{ iff } \max_i s_i^n \xrightarrow[n\rightarrow\infty]{} 0$$
- Wise crowds iff max influence vanishes
- Recall that
$$\lim_t b_j^n(t) = \sum_{i=1}^{n} s_i^n b_i^n(0) = \sum_{i=1}^n s_i^n (\mu + \varepsilon_i )= \mu + \sum_{i=1}^n s_i^n \varepsilon_i$$
- So: 
$$\plim_{n\rightarrow\infty} \left(\lim_t b_j^n(t)\right) = μ \text{ iff } \plim_{n\rightarrow\infty} \sum s_i^n \varepsilon_i =0, \text{ iff } \max_i s_i^n \xrightarrow[n\rightarrow\infty]{} 0$$



#### Reciprocal Attention:
- Suppose that $G$ is column stochastic (so each agent receives weight one). Then $s=\left(\frac{1}{n},\ldots,\frac{1}{n}\right)$ is a unit lhs eigenvector, and so $G$ is wise.
- So, reciprocal trust implies wisdom.
- But that is a very strong condition…

#### Opinion Leaders
- $s_i = \sum_j G_{ji} s_j$
- If there is some $i$ with $G_{ji} > a > 0$ for all $j$, then $s_i >a$

### Convergence without Consensus

#### Example 1

In [ ]:
trans_matrix3 = np.array([
    [0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [1/3, 0, 0, 1/3, 1/3, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 1, 0],
])
nx.draw(nx.from_numpy_array(trans_matrix3,  create_using=nx.DiGraph))

In [ ]:
b = np.array([[0], [0], [0], [0], [0], [1]])
degroot_sim(trans_matrix3, b, n=40)

#### Example 2

In [ ]:
trans_matrix4 = np.array([
    [1, 0, 0, 0, 0, 0],
    [1/4, 1/4, 1, 0, 1/4, 1/4],
    [0, 0, 1/2, 1/2, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 1/4, 0, 3/4, 0],
    [0, 0, 0, 2/3, 0, 1/3],
])
nx.draw(nx.from_numpy_array(trans_matrix4,  create_using=nx.DiGraph))

In [ ]:
b4 = np.array([[0], [0], [0], [0], [1], [0]])
degroot_sim(trans_matrix4, b4, n=40)

#### Theory

- What if our society cannot be described by an irreducible matrix $G$?
    - Can we still establish convergence? Yes
    - Do we retain consensus? No
- If a matrix is reducible there exists more than one communicating class
    - This means that for some $j \in N$ there is an $i \in N$ such that no directed path leads from $i$ to $j$
    - We can now split our network into $k$ communicating classes $B_1, B_2, \ldots, B_k$
- Recall our convergence theorem required that beliefs mixed (didn’t cycle) among everyone
- We can still guarantee this behavior within communicating classes if they are closed
- Class $B_k$ is closed if $i \in B_k$ and $T_ij > 0$ implies $j \in B_k$
- We can now hope to apply our previous result to establish consensus within each strongly connected and closed set of nodes
    - But can this guarantee convergence for the entire society?
    - Proposition (Golub & Jackson (2008)): $G$ is convergent if and only if every set of nodes that is strongly connected and
closed is aperiodic

- Example 1: Nodes {1, 2, 3}, {4}, and {5, 6} are communicating classes with only {5, 6} being closed
- Example 2: 
    - Closed sets: {1}, {3, 4, 5}, {1, 3, 4, 5}, {3, 4, 5, 6}, {1, 3, 4, 5, 6}, {1, 2, 3, 4, 5, 6}
    - Closed and strongly connected: {1} and {3, 4, 5}